In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [ ]:
if IN_COLAB:
    !git clone https://github.com/ElsayedIbrahim98/mai_project1_optimization

In [ ]:
if IN_COLAB:
    !pip3 install -r mai_project1_optimization/requirements.txt

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random
import time

if(IN_COLAB):
    from mai_project1_optimization.modules.dataset import IntelImageClassificationDataset
    from mai_project1_optimization.modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time
    from mai_project1_optimization.modules.trainer import Trainer
else:
    from modules.dataset import IntelImageClassificationDataset
    from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time
    from modules.trainer import Trainer

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

# Removed support for Tensor Units
# torch.backends.cudnn.allow_tf32 = True
# torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

In [ ]:
choice = 1 # 1,2,3
freezeLayer = False

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# SqueezeNet 1.1
if choice == 1:
    model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
    num_features = model.classifier[1].in_channels
    kernel_size = model.classifier[1].kernel_size
    if(freezeLayer):
        for param in model.parameters():
            param.requires_grad = False
    model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)


# MobileNetV2
elif choice == 2:
    model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
    num_features = model.classifier[1].in_features
    if(freezeLayer):
        for param in model.parameters():
            param.requires_grad = False
    model.classifier[1] = nn.Linear(num_features, 6)

# MobileNetV3 Small
elif choice == 3:
    model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
    num_features = model.classifier[3].in_features
    if(freezeLayer):
        for param in model.parameters():
            param.requires_grad = False
    model.classifier[3] = nn.Linear(num_features, 6)

# MobileNetV3 Large
elif choice == 4:
    model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
    num_features = model.classifier[3].in_features
    if(freezeLayer):
        for param in model.parameters():
            param.requires_grad = False
    model.classifier[3] = nn.Linear(num_features, 6)

# VisionTransformer Base 16
elif choice == 5:
    model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)
    num_features = model.heads[0].in_features
    if(freezeLayer):
        for param in model.parameters():
            param.requires_grad = False
    model.heads[0] = nn.Linear(num_features, 6)



dataloader = DataLoader(dataset.train_dataset, batch_size=128, shuffle=True)
trainer = Trainer(model=model, lr=0.001)

In [ ]:

start_train = time.time()
trainer.train(dataloader, epochs=10)
end_train = time.time()
training_time = end_train - start_train


In [ ]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

In [ ]:
session = InferenceSession(model)

# Inference output (already done in your notebook)
start_infer = time.time()
preds = session(torch.stack([item[0] for item in dataset.test_dataset]))
end_infer = time.time()
inference_time = end_infer - start_infer
true_labels = torch.tensor([item[1] for item in dataset.test_dataset])

# Classification Metrics
accuracy = Evaluator.acc(preds, true_labels).item()
precision = Evaluator.ppv(preds, true_labels).item()
recall = Evaluator.tpr(preds, true_labels).item()
f1 = Evaluator.f1_score(preds, true_labels).item()


In [ ]:
def get_memory_usage():
    import psutil, os
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1e6  # in MB

training_memory = get_memory_usage()

In [ ]:
import pandas as pd
from pathlib import Path

results = {
    "Experiment": "pin_memory + num_workers",  # customize for each run
    "Training Time (s)": round(training_time, 2),
    "Inference Time (s)": round(inference_time, 2),
    "Memory Usage (MB)": round(training_memory, 2),
    "Accuracy": round(accuracy, 4),
    "Precision": round(precision, 4),
    "Recall": round(recall, 4),
    "F1 Score": round(f1, 4)
}

# Convert to DataFrame
results_df = pd.DataFrame([results])

# Append to CSV (create if it doesn't exist)
csv_path = "/content/optimization_report.csv"
results_df.to_csv(csv_path, mode='a', header=not Path(csv_path).exists(), index=False)

print(f"✅ Results saved to: {csv_path}")